In [1]:
#!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash


In [2]:


import json
import pandas as pd
import numpy as np

from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline

spark = sparknlp.start()



22/08/28 14:17:20 WARN Utils: Your hostname, Vader resolves to a loopback address: 127.0.1.1; using 192.168.1.94 instead (on interface enp5s0f0)
22/08/28 14:17:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/sjhjrok/.ivy2/cache
The jars for the packages stored in: /home/sjhjrok/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-169f0b7b-5c6a-4cae-a0d6-4f710848a85f;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;4.1.0 in central


:: loading settings :: url = jar:file:/home/sjhjrok/anaconda3/envs/NLP/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.code.findbugs#annotations;3.0.1 in central
	found net.jcip#jcip-annotations;1.0 in central
	found com.google.code.findbugs#jsr305;3.0.1 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lombok;1.16.8 in central
	found org.slf4j#slf4j-api;1.7.21 in central
	found com.navigamez#greex;1.0 in central
	found dk.brics.automaton#automaton;1.11-8 in central
	found com.johnsnowlabs.nlp#tensorflow-cpu_2.12;0.4.3 in central
:: resolution report :: resolve 250ms :: artifacts dl 8ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.828 from 

In [7]:
text_list = [
    """Navagate to kitchen"""

]

In [8]:


documentAssembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

sentence_detector = SentenceDetector() \
    .setInputCols('document') \
    .setOutputCol('sentence')


tokenizer = Tokenizer() \
    .setInputCols(['sentence']) \
    .setOutputCol('token')

'''
Note:

For embeddings that offer multi-language support like glove_6B_300, we use 'xx'
in the language parameter.

e.g: embeddings = WordEmbeddingsModel.pretrained('glove_6B_300', lang='xx') \
        .setInputCols(["sentence", 'token']) \
        .setOutputCol("embeddings")

For dedicated embeddings like 'aner_cc_300d' the language parameter would be 
the corresponding language.

e.g: embeddings = WordEmbeddingsModel.pretrained('aner_cc_300d', lang='ar') \
        .setInputCols(["sentence", 'token']) \
        .setOutputCol("embeddings")

'''
## to use Glove embeddings
embeddings = WordEmbeddingsModel.pretrained('glove_100d', lang='en') \
        .setInputCols(["sentence", 'token']) \
        .setOutputCol("embeddings")

## to use Bert Embeddings
#embeddings = BertEmbeddings.pretrained(name='bert_base_cased', lang='en') \
#        .setInputCols(['document', 'token']) \
#        .setOutputCol('embeddings')

# select your desired model and define language
ner_model = NerDLModel.pretrained('nerdl_snips_100d') \
    .setInputCols(['sentence', 'token', 'embeddings']) \
    .setOutputCol('ner')

ner_converter = NerConverter() \
    .setInputCols(['sentence', 'token', 'ner']) \
    .setOutputCol('ner_chunk')

nlp_pipeline = Pipeline(stages=[
    documentAssembler, 
    sentence_detector,
    tokenizer,
    embeddings,
    ner_model,
    ner_converter
])


glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]
nerdl_snips_100d download started this may take some time.
Approximate size to download 13.7 MB
[OK!]


In [9]:

empty_df = spark.createDataFrame([['']]).toDF('text')
pipeline_model = nlp_pipeline.fit(empty_df)
df = spark.createDataFrame(pd.DataFrame({'text': text_list}))
result = pipeline_model.transform(df)








In [10]:

from sparknlp_display import NerVisualizer

NerVisualizer().display(
    result = result.collect()[0],
    label_col = 'ner_chunk',
    document_col = 'document'
)




